<a href="https://colab.research.google.com/github/OneFineStarstuff/Onefinebot/blob/main/Example_Style_Transfer_with_Deep_Learning_(using_PyTorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

import torch
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt

# Load images (content and style)
content_img = Image.open("/content/drive/My Drive/Colab Notebooks/your_uploaded_content_image.jpg").convert('RGB')
style_img = Image.open("/content/drive/My Drive/Colab Notebooks/your_uploaded_style_image.jpg").convert('RGB')

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])
content_tensor = transform(content_img).unsqueeze(0)
style_tensor = transform(style_img).unsqueeze(0)

# Load a pre-trained VGG model for style transfer
vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).features.eval()

# Function to compute content loss
def content_loss(content, target):
    return torch.nn.functional.mse_loss(content, target)

# Function to compute style loss
def gram_matrix(input):
    batch_size, feature_maps, h, w = input.size()
    features = input.view(batch_size * feature_maps, h * w)
    G = torch.mm(features, features.t())
    return G.div(batch_size * feature_maps * h * w)

def style_loss(style, target):
    G = gram_matrix(target)
    A = gram_matrix(style)
    return torch.nn.functional.mse_loss(G, A)

# Function to compute style transfer
def style_transfer(content, style, model, num_steps=500, style_weight=1e6, content_weight=1):
    input_img = content.clone()
    optimizer = torch.optim.LBFGS([input_img.requires_grad_()])

    content_features = model(content).detach()
    style_features = model(style).detach()

    for step in range(num_steps):
        def closure():
            input_img.data.clamp_(0, 1)
            optimizer.zero_grad()
            input_features = model(input_img)
            c_loss = content_weight * content_loss(input_features, content_features)
            s_loss = style_weight * style_loss(style_features, input_features)
            loss = c_loss + s_loss
            loss.backward()
            return loss
        optimizer.step(closure)

    return input_img

# Apply style transfer and display
output = style_transfer(content_tensor, style_tensor, vgg)
plt.imshow(output.squeeze().permute(1, 2, 0).detach().numpy())
plt.show()